In [1]:
## 聚类算法

In [1]:
import math

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs

In [3]:
n_samples = 1500
random_state = 170
X, y = make_blobs(n_samples=n_samples, random_state=random_state)

In [4]:
xdf = pd.DataFrame(X)
xdf.columns = ['a', 'b']

In [5]:
y

array([1, 1, 0, ..., 2, 2, 2])

In [6]:
xdf.columns = [f'f{i}' for i in range(len(list(xdf)))]

In [7]:
xdf

,f0,f1
0,-5.198113,0.641869
1,-5.752295,0.418627
2,-10.844898,-7.553523
3,-4.570985,-0.801017
4,-3.519162,0.039345
5,1.603916,0.763880
6,-9.751574,-5.203026
7,-11.510236,-4.162843
8,-7.726758,-5.866566
9,2.676567,3.298728


In [39]:
def cal_nearest_center(xs, centers, p=2):
    n = len(xs)
    best_center = None
    min_dist = 0
    for i, line in enumerate(centers):
        s = sum([math.pow(abs(xs[j] - line[j]), p) for j in range(n)])
        dis = math.pow(s, 1/p)
        if best_center is None or dis < min_dist:
            best_center = i
            min_dist = dis
    return [best_center, min_dist]

In [19]:
class Kmeans:
    def __init__(self, n_clusters=4, max_iter=100, random_state=None):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.random_state = random_state
        self.run_iter = 0
        ###
        self.centers = []
        self.sse = 0
    
    def run(self, x, p=2):
        xshape = len(x.columns)
        x = x.copy()
        x.columns = [f'f{i}' for i in range(len(list(xdf)))]
        if self.random_state:
            samples = x.sample(n=self.n_clusters, random_state=self.random_state)
        else:
            samples = x.sample(n=self.n_clusters)
        ####
        ndf = x.copy()
        ndf['tag'] = -1
        ndf['last_tag'] = -2
        ndf['dis'] = 0
        d = list(x[x.index.isin(samples.index)].apply(lambda row: list(row), axis=1))
        ####
        self.run_iter = 0
        while self.run_iter < self.max_iter and len(ndf[ndf.tag != ndf.last_tag]) > 0:
            self.run_iter += 1
            ### 划分簇
            ndf['last_tag'] = ndf.tag
            ret = np.array(list(ndf.apply(lambda row: cal_nearest_center(list(row)[:xshape], list(d), p), axis=1) )).T
            ndf['tag'] = ret[0]
            ndf['dis'] = ret[1]
            ###
            self.sse = sum(ndf.dis)
            ### 更新簇中心
            for i, line in enumerate(d):
                _ = ndf[ndf.tag == i].to_dict()
                for j in range(len(line)):
                    line[j] = np.mean(list(_[f'f{j}'].values()))
            self.centers = d 
        return ndf.tag
kmeans = Kmeans(n_clusters=3)
tags = kmeans.run(xdf)

In [12]:
import plotly.graph_objects as go

def draw_scatters(xs, ys, names):
    fig = go.Figure()
    # Add traces
    for x, y, name in zip(xs, ys, names):
        fig.add_trace(go.Scatter(x=x, y=y,
                    mode='markers',
                    name=name))
    fig.show()

In [72]:
kmeans = Kmeans(n_clusters=3)
tags = kmeans.run(xdf)
xdf['tag'] = tags
xs = []
ys = []
names = []
for tag in set(tags):
    xs.append(xdf[xdf.tag == tag].f0)
    ys.append(xdf[xdf.tag == tag].f1)
    names.append(tag)
###
names.append('center')
xs.append([line[0] for line in kmeans.centers])
ys.append([line[1] for line in kmeans.centers])

draw_scatters(xs, ys, names)

In [70]:
kmeans = Kmeans(n_clusters=4)
tags = kmeans.run(xdf)
xdf['tag'] = tags
xs = []
ys = []
names = []
for tag in set(tags):
    xs.append(xdf[xdf.tag == tag].f0)
    ys.append(xdf[xdf.tag == tag].f1)
    names.append(tag)
###
names.append('center')
xs.append([line[0] for line in kmeans.centers])
ys.append([line[1] for line in kmeans.centers])

draw_scatters(xs, ys, names)

In [23]:
kmeans = Kmeans(n_clusters=5)
tags = kmeans.run(xdf)
xdf['tag'] = tags
xs = []
ys = []
names = []
for tag in set(tags):
    xs.append(xdf[xdf.tag == tag].f0)
    ys.append(xdf[xdf.tag == tag].f1)
    names.append(tag)
###
names.append('center')
xs.append([line[0] for line in kmeans.centers])
ys.append([line[1] for line in kmeans.centers])

draw_scatters(xs, ys, names)

In [91]:
kmeans = Kmeans(n_clusters=5, max_iter=10)
tags = kmeans.run(xdf)
xdf['tag'] = tags
xs = []
ys = []
names = []
for tag in set(tags):
    xs.append(xdf[xdf.tag == tag].f0)
    ys.append(xdf[xdf.tag == tag].f1)
    names.append(tag)
draw_scatters(xs, ys, names)

In [92]:
kmeans = Kmeans(n_clusters=6, max_iter=10)
tags = kmeans.run(xdf)
xdf['tag'] = tags
xs = []
ys = []
names = []
for tag in set(tags):
    xs.append(xdf[xdf.tag == tag].f0)
    ys.append(xdf[xdf.tag == tag].f1)
    names.append(tag)
draw_scatters(xs, ys, names)